# Reporte de problemas fitosanitarios en plantaciones de agave
--------------------

## Equipo 36

| Nombre | Matrícula |
| ------ | --------- |
| André Martins Cordebello | A00572928 |
| Enrique Eduardo Solís Da Costa | A00572678 |
| Delbert Francisco Custodio Vargas | A01795613 |

## Avance 2: Feature engineering

- Crear nuevas características para mejorar el rendimiento de los modelos.
- Mitigar el riesgo de características sesgadas y acelerar la convergencia de algunos algoritmos.

# Feature Engineering

## EDA

Durante el avance 1 fue posible comprender ciertos comportamientos de nuestro dataset, así como la selección de columnas que sí aportaban información para entender de mejor manera las características de infección de gorgojos del agave en predios del mismo.

Con esto, usando como base los datasets compartidos por la CNIT fue posible obtener el dataset llamada `all_historic_captures.xlsx`, el cual contiene información del 2014 a Agosto de 2025 sobre el nivel de incidencia o infección encontrado  en predios de agave. Estas muestras se obtuvieron con base a las condiciones definidas en el **Manual Operativo de la campaña contra plagas reglamentadas del Agave**, disponible en https://www.gob.mx/cms/uploads/attachment/file/234136/Manual_Operativo_de_la_campa_a_contra_plagas_reglamentadas_del_agave_2017.compressed.pdf, lo cual fue confirmado por nuestro Sponsor (CNIT).

Por lo anterior, el objetivo principal de este proyecto integrador es el desarrollo de un ChatBot el cual:

- Pueda recibir reportes que incluyan ubicación, breve descripción de lo encontrado, fotografías y clasificación de riesgo de parte del cuerpo técnico y ciudadanía en general.
- Este ChatBot también debe tener la capacidad de responder y alertar a los usuarios sobre focos de infección reportados o confirmados en las cercanías.

La recepción de esta información es importante ya que permitirá tomar decisiones sobre lo que las brigadas de desinfección deben atacar primero.

Por lo anterior, durante nuestro EDA (análisis exploratorio) encontramos los siguientes hallazgos:

- La captura de gorgojos del agave en las trampas tiende a aumentar en épocas lluviosas.
- Los focos de infección severos y moderados tienen una dispersión geográfica menor.
- No es normal encontrar predios de más de 8 años de antiguedad, lo que nos da una idea de cuáles predios podrían estar posiblemente abandonados.
- La mayor densidad de trampas se encuentra en el estado de Jalisco.
- No es normal encontrar focos severos de infección, pero la presencia de éstos aumenta en la época lluviosa. Es posible confirmar que los casos severos de infección son casos atípicos, ya que el valor del percentil 95 se encuentra en 17.5 capturas por trampa.
- Durante la pandemia (2020 a 2024 aproximadamente), el muestreo de las trampas colocados no fue tan constante como en años posteriores o previos a la pandemia. Esto causa un efecto de sesgo en nuestro dataset.


Y al transformar un poco nuestro dataset obtuvimos estas columnas finales:

| Feature  | Tipo | Notas |
| -------  | ---- | ----- |
| tramp_id | object | Es el ID único que se le da a la trampa al colocarse en alguna parcela o predio. Una misma trampa puede colocarse en distintos predios, pero la identificación de la misma cambia acorde a dónde se colocó. |
| sampling_date | datetime64[ns] | Es la fecha en la que se llevó a cabo el conteo de cadáveres de gorgojo en la trampa.|
| lat y lon | float64 | La `latitud` y `longitud` permiten conocer la ubicación de donde se llevó a cabo el muestreo. |
| municipality | object | Municipio al que pertenecía la trampa al momento de hacer el muestreo. |
| square_area | float64 | Area que cubre el predio donde se colocó la trampa. |
| plantation_age | float64 | Años que una plantación de agave tiene desde la última purga. |
| capture_count | float64 | Cantidad de gorgojos del agave encontrados dentro de la trampa. |
| state | object | Estado de México donde se encontraba la trampa colocada. |
| severity | object | lLa severidad de la infestación encontrada durante el muestreo. Estos niveles fueron definidos por la SICAFI.|
| Month | int32 | Mes en que se llevó a cabo el muestreo de la trampa. |
| Year | int32 | Año en que se llevó a cabo el muestreo de la trampa. |
| MonthName | object | Nombre del   mes en que se llevó a cabo el muestreo de la trampa. |
| MonthYear | datetime64[ns] |  Combinación del año y mes en que se llevó a cabo el muestreo de la trampa. Esta ya es una característica creada a partir de otras columnas del Dataset. |

Con esto, estaremos trabajando la ingeniería de característiscas con base en estas columnas.

### Librerías  a importar

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os


plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("Set2")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

### Características generales del dataset final

In [ ]:
# Carga de dataset
all_historic_captures_df = pd.read_excel( "all_historic_captures.xlsx", sheet_name="historic_captures", header= 0)

In [3]:
# Eliminamos las columnas que usamos para llevar a cabo una mejor comprensión en el EDA,
# esto porque debemos decidir más adelante si serán necesarias para nuestro proceso de FE.
all_historic_captures_df.drop(labels=['Month', 'Year', 'MonthName', 'MonthYear', 'plantation_age_group', 'surface_group', 'no_area'],axis=1, inplace=True)

all_historic_captures_df.head(5)

,tramp_id,sampling_date,lat,lon,municipality,square_area,plantation_age,capture_count,state,severity
0,146_THUE13-11-023-0637T01,2022-07-21,20.401984,-101.702729,PÉNJAMO,1.75,0,8.0,GUANAJUATO,1-25
1,146_THUE13-11-023-0637T01,2022-04-19,20.401984,-101.702729,PÉNJAMO,1.75,0,5.0,GUANAJUATO,1-25
2,146_THUE13-11-023-0637T01,2022-02-25,20.401984,-101.702729,PÉNJAMO,1.75,0,5.0,GUANAJUATO,1-25
3,146_THUE13-11-023-0637T01,2022-02-14,20.401984,-101.702729,PÉNJAMO,1.75,0,4.0,GUANAJUATO,1-25
4,146_THUE13-11-023-0637T01,2022-05-18,20.401984,-101.702729,PÉNJAMO,1.75,0,6.0,GUANAJUATO,1-25


In [4]:
all_historic_captures_df.describe(include='number').T

,count,mean,std,min,25%,50%,75%,max
lat,827856.0,20.642391,0.728798,18.987685,20.161786,20.597498,20.884595,23.439659
lon,827856.0,-102.755748,1.301210,-114.108837,-103.721093,-102.650350,-102.184024,-98.662612
square_area,827856.0,5.020235,13.905933,0.000000,0.000000,0.510000,3.935500,787.820000
plantation_age,827856.0,2.428498,1.825808,0.000000,1.000000,2.000000,4.000000,19.000000
capture_count,827856.0,4.264352,8.156295,0.000000,0.000000,2.000000,5.000000,427.000000


In [5]:
all_historic_captures_df.describe(exclude='number').T

,count,unique,top,freq,mean,min,25%,50%,75%,max
tramp_id,827856.0,84063.0,716.0,125.0,NaN,NaN,NaN,NaN,NaN,NaN
sampling_date,827856,NaN,NaN,NaN,2018-12-29 19:19:48.102277888,2014-01-01 00:00:00,2016-05-19 00:00:00,2017-11-10 00:00:00,2021-12-28 00:00:00,2025-08-31 00:00:00
municipality,827856,149,ARANDAS,61191,NaN,NaN,NaN,NaN,NaN,NaN
state,827856,6,JALISCO,442628,NaN,NaN,NaN,NaN,NaN,NaN
severity,827856,4,1-25,518659,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
all_historic_captures_df.isna().sum() / len(all_historic_captures_df) * 100

tramp_id          0.0
sampling_date     0.0
lat               0.0
lon               0.0
municipality      0.0
square_area       0.0
plantation_age    0.0
capture_count     0.0
state             0.0
severity          0.0
dtype: float64

### `Feature engineering` sobre `square_area`

Al revisar nuevamente los valores de `square_area` (o `Superficie (ha)` originalmente), es posible notar que casi el 48% de éstos es igual a `0.00`. Esto, como discutimos con anterioridad, indica que durante el proceso de muestreo de años previos a 2024 y 2025 no hubo un control de calidad para mitigar la falta de información en esta columna. Esto fue confirmado por nuestro Sponsor.

Pero como consideramos que esta información es importante, hemos decidido trabajar la siguiente estrategia de imputación:

- Reducir la cantidad de valores `0.00` por medio del ID de cada trampa. Como el ID de la trampa está en función del predio muestreado, es posible asumir que con que un ID de trampa contenga información del área, esta se puede replicar a las demás instancias de ese mismo ID de trampa.

- Luego de esto, usaremos un algoritmo conocido como K-NN (K nearest neighbour) para verificar cuales trampas con un registro de superficie distinto a `0.00` se encuentran cerca de otras trampas con un área distinta a `0.00`. La distancia entre éstas trampas debe ser entre 100m a 500m según el manual de operación.

- Por último, analizaremos cuántos registros quedan pendientes aún de tener un valor distinto a `0.00` y decidiremos más adelante qué hacer con ellos.

In [95]:
df = all_historic_captures_df

print(f"Trampas registradas con área mayor a 0.00: { (df['square_area'] > 0 ).sum()}")
print(f"Trampas registradas con área igual a 0.00: { (df['square_area'] == 0 ).sum()}")


Trampas registradas con área mayor a 0.00: 428434
Trampas registradas con área igual a 0.00: 399422


##### Usando el ID de trampa para llenar valores vacíos

In [96]:
# Hacemos una agregacion de los datos de area para los registros que tienen un valor mayor a 0
trap_area_stats = df[df['square_area'] > 0].groupby('tramp_id')['square_area'].agg([
    'mean', 
    'median', 
    'std', 
    'count'
]).reset_index()

# Simplificamos las columnas
trap_area_stats.columns = ['tramp_id', 'mean_area', 'median_area', 'std_area', 'count']

print(f"Trampas únicas con área conocida: {len(trap_area_stats):,}")

print(f"\nEstadística de trampas unicas con area:")
print(trap_area_stats.head(10))

traps_with_zero_std = (trap_area_stats['std_area'] == 0).sum()
print(f"Traps with std = 0: {traps_with_zero_std:,}")

Trampas únicas con área conocida: 46,233

Estadística de trampas unicas con area:
  tramp_id  mean_area  median_area  std_area  count
0        1     5.9549       5.9549       0.0     21
1        2     2.4495       2.4495       0.0     17
2        3     2.4152       2.4152       0.0     21
3        4     4.5461       4.5461       0.0     17
4        5     3.8173       3.8173       0.0     17
5        6     2.0322       2.0322       0.0     17
6        7     9.8021       9.8021       0.0     16
7        8     6.1943       6.1943       0.0     20
8        9     4.2533       4.2533       0.0     21
9       10     4.2300       4.2300       0.0     19
Traps with std = 0: 28,511


In [97]:

# Trampas con area igual a 0.00
missing_area = df['square_area'] == 0

# Cruzamos las trampas con area 0.00 vs. trampas con el mismo ID que tienen area distinta a 0.00
can_fill = df[missing_area]['tramp_id'].isin(trap_area_stats['tramp_id'])
print( f"Cantidad de registros de `square_area` que podemos rellenar: {len(df[missing_area][can_fill])}" )

Cantidad de registros de `square_area` que podemos rellenar: 79287


In [98]:
# Procedemos a crear nuevas columnas para mantener los valores originales de square_area.
# Columna para agregar el valor calculado con la estrategia 1

df['square_area_imputed'] = df['square_area'].copy()
df['imputation_method'] = 'original'  
df['imputation_method'] = df['imputation_method'].where(
    df['square_area'] > 0, 'none'
)

# Usamos un diccionario y una mascara para modificar solo los registros que necesitamos
trap_median_map = trap_area_stats.set_index('tramp_id')['median_area'].to_dict()

# La Mascara toma en cuenta los registros con area igual a 0.00 y el index de cada uno y lo relaciona al index en el diccionario
mask = (df['square_area'] == 0) & (df['tramp_id'].isin(trap_median_map.keys()))

# Cambiamos el valor del area calculada (que es 0.00 para estos casos) y el origen o valor en imputation_method para dejar una bandera
df.loc[mask, 'square_area_imputed'] = df.loc[mask, 'tramp_id'].map(trap_median_map)
df.loc[mask, 'imputation_method'] = 'same_trap_id'

# Imprimimos la cantidad de registros que modificamos
print(f"Se llenaron { len(df[ df['imputation_method'] == 'same_trap_id' ]) } registros usando el mismo ID de la trampa.")

Se llenaron 79287 registros usando el mismo ID de la trampa.


##### Ahora usaremos KNN para rellenar el resto de información

- `original`: son todas los registros de area que originalmente son distintos a 0.00
- `none`: son los registros que aún no hemos imputado
- `samte_trap_id`: son los registros que tienen un valor imputado en la columna `square_area_imputed` por medio de usar el `trap_id`.

In [99]:
print("\nOrigen de los valores de la columna square_area_imputed:")
print(df['imputation_method'].value_counts())


Origen de los valores de la columna square_area_imputed:
imputation_method
original        428434
none            320135
same_trap_id     79287
Name: count, dtype: int64


In [100]:
from sklearn.neighbors import KNeighborsRegressor
import warnings

# Ignoraremos warnings
warnings.filterwarnings('ignore')

# No debemos  sobre-escribir lo que ya imputamos anterioremente
# Entonces usaremos esos indexes como mascara
train_mask = df['imputation_method'].isin(['original', 'same_trap_id'])

# Tomamos los indexes que ya sabemos que en teoría están bien
X_train = df[train_mask][['lat', 'lon']].values
y_train = df[train_mask]['square_area_imputed'].values

# Nuestro test serán los registros de los que sabemos que tienen 0.00 en square_area
# y que no han sido imputados por el paso previo.
test_mask = df['imputation_method'] == 'none'
X_test = df[test_mask][['lat', 'lon']].values

print(f"Registros a predecir su quare_area: {len(X_test):,}")

Registros a predecir su quare_area: 320,135


In [ ]:
# Usaremos Harversine para tomar en cuenta la curvatura de la tierra en la distancia que usemos para conocer cuales trampas están
# cerca unas de otras, y se necesita que las longitudes y latitudes estén en radianes
X_train_rad = np.radians(X_train)
X_test_rad = np.radians(X_test)

knn = KNeighborsRegressor(
    n_neighbors=5,
    weights='distance',  # Weight by inverse distance
    metric='haversine',  # Geographic distance
    n_jobs=-1           # Use all available cores
)

knn.fit(X_train_rad, y_train)